# Notebook 2: Least Squares Regression and Scaling

#TODO What is happening here

In [9]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import glob
import pandas as pd
import sklearn
from src import ana_utils as utils

#np.set_printoptions(suppress=True)
from sklearn.linear_model import LinearRegression

np.random.seed(4)

Import datasets that were preprocessed in Notebook 1

In [2]:
wb_data = pd.read_csv("data/wb_data.csv", index_col="Country Name")
wb_data_short = pd.read_csv("data/wb_data_short.csv", index_col="Country Name")
whr_data = pd.read_csv("data/whr_data.csv", index_col="Country name")

# test: are the same countries present in each dataset?
print(sorted(list(wb_data.index))==sorted(list(whr_data.index)))

True


Split data into train and test set. We choose a 80/20 split, i.e. 120 countries in the training set and 30 countries in the test set.

In [3]:
test_size = 30
train, test, train_gt, test_gt = utils.split_data(wb_data_short, whr_data, test_size)

# verify set shapes
print(train.shape, test.shape, train_gt.shape, test_gt.shape)

#verify that data prder and groundtruth order and indices match 
print(list(train.index)==list(train_gt.index), list(test.index)==list(test_gt.index))


(120, 120) (30, 120) (120, 1) (30, 1)
True True


## Linear regression


Let's see how linear least squares regression performs on wb_data and wb_data_short (redundant indicators removed). We choose 2000-fold validation after noticing quite some variance for lower n.

In [4]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="no_scaling", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="no_scaling", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 15.057832109334568
Mean train loss (reduced set of indicators): 0.00013509184869362696
Adjusted R^2: 1.000351358046903
The average size of the first ten coefficients ((full set of indicators)): [ 0.0153 -0.0084 -0.     -0.     -0.0029  0.0544 -0.0256  0.08    0.0953
 -0.0874] 

Mean test loss (reduced set of indicators): 5.392762624045163
Mean train loss (reduced set of indicators): 0.0465786384326334
Adjusted R^2: 5.478543217680773
The average size of the first ten coefficients (reduced set of indicators): [ 0.0202 -0.0062  0.     -0.     -0.0056  0.4887  0.1943  0.0929 -0.0216
  0.0127]


While linear regression performs better after manually removing redundancies, both of the results are still quite poor. 
We suspect multicolinearity to be a main reason for bad performance. 

But before starting to deal with multicolinearity, we want to normalize/standardize the data. This is because, in the end, we aim to compare coefficients. But as of now, coefficient sizes vary drastically, thus requiring rescaling of the features.

Hence, performing analysis also on the normalized/standardized data along the way is necessary to prevent us from developing a model that works only on non-normalized/non-standardized data. 

### Normalization 

Here, we normalize each row, using the L2 norm. That is, for each country the indicator values are scaled such that the sum of all squared indicator values is one.

In [11]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="normalize", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="normalize", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 22.838530795603155
Mean train loss (reduced set of indicators): 2.6258441234259437e-27
Adjusted R^2: 1.0
The average size of the first ten coefficients ((full set of indicators)): [  8.4407  -2.2765  -0.1456  -0.6854  -0.6961  29.4861  10.7554  16.4411
   0.6126 -12.8363] 

Mean test loss (reduced set of indicators): 5.439114774171314
Mean train loss (reduced set of indicators): 0.07630907271271567
Adjusted R^2: 8.37538507346491
The average size of the first ten coefficients (reduced set of indicators): [ 6.2402 -1.1786  1.2447 -0.2188 -2.475  57.9424 50.6566  7.1706 -4.0273
  0.9236]


### Standardization
Each value x is scaled with the formula $\frac{x-\mu}{\sigma}$

In [6]:
least_squares = sklearn.linear_model.LinearRegression()

# full set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data, gt=whr_data, test_size=test_size, scaling="standardize", calc_adj_r_squared=True)
print("Mean loss (full set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients ((full set of indicators)):", avg_coefs[:10], "\n")

# manually reduced set
loss_list, mean_loss, mean_train_loss, coef_list, avg_coefs, adjusted_r_squared = utils.n_fold_ceval(reg_model=least_squares, n=1000, data=wb_data_short, gt=whr_data, test_size=test_size, scaling="standardize", calc_adj_r_squared=True)
print("Mean test loss (reduced set of indicators):", mean_loss)
print("Mean train loss (reduced set of indicators):", mean_train_loss)
print("Adjusted R^2:", adjusted_r_squared)
print("The average size of the first ten coefficients (reduced set of indicators):", avg_coefs[:10])

Mean loss (full set of indicators): 22.745297550915108
Mean train loss (reduced set of indicators): 2.7836955077484906e-27
Adjusted R^2: 1.0
The average size of the first ten coefficients ((full set of indicators)): [ 0.6407 -0.1686 -0.0085 -0.0511 -0.0599  2.213   0.8443  1.2166  0.2567
 -0.9821] 

Mean test loss (reduced set of indicators): 5.538178956177356
Mean train loss (reduced set of indicators): 0.07093305524307483
Adjusted R^2: 7.849987267967027
The average size of the first ten coefficients (reduced set of indicators): [ 0.523  -0.1057  0.0993 -0.0078 -0.1815  4.8282  4.0798  0.6879 -0.3377
  0.0875]


### Conclusion for Standardization/Normalization
Both methods seem to worsen mean loss for the regression on the full dataset. However, they don't change the mean loss notably for the dataset containing the manually reduced set of indicators.  
Therefore, we will from now on normalize the data in order to make the resulting regression coefficients more interpretable.